In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')

In [2]:
df_train= pd.read_csv('data/Train_pjb2QcD.csv', header=0)

In [3]:
X = df_train.drop(['ID','Applicant_BirthDate', 'Business_Sourced', 'Applicant_City_PIN', 'Manager_DoB'], axis=1)
y = df_train['Business_Sourced']

In [4]:
X['Applicant_Gender'] = X['Applicant_Gender'].fillna( X.Applicant_Gender.dropna().mode().values[0])
X['Applicant_Gender'] = X['Applicant_Gender'].map({'F':0, 'M':1}).astype(np.int)

X['Applicant_Marital_Status'] = X['Applicant_Marital_Status'].fillna( X.Applicant_Marital_Status.dropna().mode().values[0])
X['Applicant_Marital_Status'] = X['Applicant_Marital_Status'].map({'S':0, 'M':1})

X['Manager_Gender'] = X['Manager_Gender'].fillna( X.Manager_Gender.dropna().mode().values[0])
X['Manager_Gender'] = X['Manager_Gender'].map({'F':0, 'M':1})

X['Manager_Status'] = X['Manager_Status'].fillna( X.Manager_Status.dropna().mode().values[0])
X['Manager_Status'] = X['Manager_Status'].map({'Probation':0, 'Confirmation':1}).astype(np.int)

X['Applicant_Occupation'] = X['Applicant_Occupation'].fillna( X.Applicant_Occupation.dropna().mode().values[0])
X['Applicant_Occupation'] = X['Applicant_Occupation'].map({'Others':0, 'Business':1, 'Salaried':2,
                                                           'Self Employed':3,'Student':4}).astype(np.int)

X['Applicant_Qualification'] = X['Applicant_Qualification'].fillna( X.Applicant_Qualification.dropna().mode().values[0])
X['Applicant_Qualification'] = X['Applicant_Qualification'].map({'Graduate':0,
                                                                 'Class XII':1,
                                                                 'Class X':2, 
                                                                 'Masters of Business Administration':3,
                                                                 'Associate/Fellow of Institute of Company Secretories of India':4,
                                                                 'Associate / Fellow of Institute of Chartered Accountans of India':5,
                                                                 'Others':6,
                                                                 'Associate/Fellow of Insurance Institute of India':7,
                                                                 'Associate/Fellow of Acturial Society of India':8,
                                                                 'Certified Associateship of Indian Institute of Bankers':9,
                                                                 'Professional Qualification in Marketing':10}).astype(np.int)


X['Manager_Joining_Designation'] = X['Manager_Joining_Designation'].fillna( X.Manager_Joining_Designation.dropna().mode().values[0])
X['Manager_Joining_Designation'] = X['Manager_Joining_Designation'].map({'Other':0, 'Level 1':1, 'Level 2':2,
                                                                         'Level 3':3, 'Level 4':4, 'Level 5':5,
                                                                         'Level 6':6, 'Level 7':7}).astype(np.int)

X['Manager_Current_Designation'] = X['Manager_Current_Designation'].fillna( X.Manager_Current_Designation.dropna().mode().values[0])
X['Manager_Current_Designation'] = X['Manager_Current_Designation'].map({'Level 1':1, 'Level 2':2, 'Level 3':3,
                                                                         'Level 4':4, 'Level 5':5}).astype(np.int)

In [5]:
X['Manager_Num_Application'] = X['Manager_Num_Application'].fillna( X.Manager_Num_Application.dropna().mode().values[0]).astype(np.int)
X['Manager_Num_Coded'] = X['Manager_Num_Coded'].fillna( X.Manager_Num_Coded.dropna().mode().values[0]).astype(np.int)
X['Manager_Num_Products'] = X['Manager_Num_Products'].fillna( X.Manager_Num_Products.dropna().mode().values[0]).astype(np.int)
X['Manager_Num_Products2'] = X['Manager_Num_Products2'].fillna( X.Manager_Num_Products2.dropna().mode().values[0]).astype(np.int)
X['Manager_Business'] = X['Manager_Business'].fillna( X.Manager_Business.dropna().mean()).astype(np.int)
X['Manager_Business2'] = X['Manager_Business2'].fillna( X.Manager_Business2.dropna().mean()).astype(np.int)

X['Manager_Grade'] = X['Manager_Grade'].fillna( X.Manager_Grade.dropna().mode().values[0]).astype(np.int)
X['Applicant_Receipt_Month'] = [t.month for t in pd.DatetimeIndex(X.Application_Receipt_Date)]
X['Manager_Join_Year'] = [t.year for t in pd.DatetimeIndex(X.Manager_DOJ)]
X['Manager_Join_Month'] = [t.month for t in pd.DatetimeIndex(X.Manager_DOJ)]
X= X.drop(['Application_Receipt_Date', 'Manager_DOJ'], axis=1)
X['Manager_Join_Year'] = X['Manager_Join_Year'].fillna( X.Manager_Join_Year.dropna().mean()).astype(np.int)
X['Manager_Join_Month'] = X['Manager_Join_Month'].fillna( X.Manager_Join_Month.dropna().mean()).astype(np.int)

X['Applicant_Marital_Status'] = X['Applicant_Marital_Status'].fillna( X.Applicant_Marital_Status.dropna().mode().values[0])

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['Office_PIN'] = le.fit_transform(X['Office_PIN'])

In [6]:
from sklearn.preprocessing import StandardScaler
sle = StandardScaler()
X_std = sle.fit_transform(X)

In [7]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion = 'entropy', n_estimators=500, oob_score=True, random_state=1, n_jobs=-1)
forest.fit(X_std, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [8]:
def repredict(est,t, xtest):
    probs=est.predict_proba(xtest)
    p0 = probs[:,0]
    p1 = probs[:,1]
    ypred = (p1 >= t)*1
    return ypred

In [9]:
df_test = pd.read_csv('data/Test_wyCirpO.csv')

In [10]:
test_X = df_test.drop(['ID','Applicant_BirthDate', 'Applicant_City_PIN', 'Manager_DoB'], axis=1)

In [11]:
test_X['Applicant_Gender']= test_X['Applicant_Gender'].fillna( test_X['Applicant_Gender'].dropna().mode().values[0])
test_X['Applicant_Gender'] = test_X['Applicant_Gender'].map({'F':0, 'M':1})

test_X['Applicant_Marital_Status'] = test_X['Applicant_Marital_Status'].fillna(test_X.Applicant_Marital_Status.dropna().mode().values[0])
test_X['Applicant_Marital_Status'] = test_X['Applicant_Marital_Status'].map({'S':0, 'M':1})

test_X['Manager_Gender'] = test_X['Manager_Gender'].fillna( test_X.Manager_Gender.dropna().mode().values[0])
test_X['Manager_Gender'] = test_X['Manager_Gender'].map({'F':0, 'M':1})

test_X['Manager_Status'] = test_X['Manager_Status'].fillna( test_X.Manager_Status.dropna().mode().values[0])
test_X['Manager_Status'] = test_X['Manager_Status'].map({'Probation':0, 'Confirmation':1})

test_X['Applicant_Occupation'] = test_X['Applicant_Occupation'].fillna( test_X.Applicant_Occupation.dropna().mode().values[0])
test_X['Applicant_Occupation'] = test_X['Applicant_Occupation'].map({'Others':0, 'Business':1, 'Salaried':2,
                                                           'Self Employed':3,'Student':4})

test_X['Applicant_Qualification'] = test_X['Applicant_Qualification'].fillna( test_X.Applicant_Qualification.dropna().mode().values[0])
test_X['Applicant_Qualification'] = test_X['Applicant_Qualification'].map({'Graduate':0,
                                                                 'Class XII':1,
                                                                 'Class X':2, 
                                                                 'Masters of Business Administration':3,
                                                                 'Associate/Fellow of Institute of Company Secretories of India':4,
                                                                 'Associate / Fellow of Institute of Chartered Accountans of India':5,
                                                                 'Others':6,
                                                                 'Associate/Fellow of Insurance Institute of India':7,
                                                                 'Associate/Fellow of Acturial Society of India':8,
                                                                 'Certified Associateship of Indian Institute of Bankers':9,
                                                                 'Professional Qualification in Marketing':10})


test_X['Manager_Joining_Designation'] = test_X['Manager_Joining_Designation'].fillna( test_X.Manager_Joining_Designation.dropna().mode().values[0])
test_X['Manager_Joining_Designation'] = test_X['Manager_Joining_Designation'].map({'Other':0, 'Level 1':1, 'Level 2':2,
                                                                         'Level 3':3, 'Level 4':4, 'Level 5':5,
                                                                         'Level 6':6, 'Level 7':7})

test_X['Manager_Current_Designation'] = test_X['Manager_Current_Designation'].fillna( test_X.Manager_Current_Designation.dropna().mode().values[0])
test_X['Manager_Current_Designation'] = test_X['Manager_Current_Designation'].map({'Level 1':1, 'Level 2':2, 'Level 3':3,
                                                                         'Level 4':4, 'Level 5':5})

In [12]:
test_X['Manager_Num_Application'] =test_X['Manager_Num_Application'].fillna( test_X.Manager_Num_Application.dropna().mode().values[0]).astype(np.int)
test_X['Manager_Num_Coded'] = test_X['Manager_Num_Coded'].fillna( test_X.Manager_Num_Coded.dropna().mode().values[0]).astype(np.int)
test_X['Manager_Num_Products'] = test_X['Manager_Num_Products'].fillna( test_X.Manager_Num_Products.dropna().mode().values[0]).astype(np.int)
test_X['Manager_Num_Products2'] = test_X['Manager_Num_Products2'].fillna( test_X.Manager_Num_Products2.dropna().mode().values[0]).astype(np.int)
test_X['Manager_Business'] = test_X['Manager_Business'].fillna( test_X.Manager_Business.dropna().mean()).astype(np.int)
test_X['Manager_Business2'] = test_X['Manager_Business2'].fillna( test_X.Manager_Business2.dropna().mean()).astype(np.int)

In [13]:
test_X['Manager_Grade'] = test_X['Manager_Grade'].fillna( test_X.Manager_Grade.dropna().mode().values[0]).astype(np.int)

In [14]:
test_X['Applicant_Receipt_Month'] = [t.month for t in pd.DatetimeIndex(test_X.Application_Receipt_Date)]
test_X['Manager_Join_Year'] = [t.year for t in pd.DatetimeIndex(test_X.Manager_DOJ)]
test_X['Manager_Join_Month'] = [t.month for t in pd.DatetimeIndex(test_X.Manager_DOJ)]
test_X= test_X.drop(['Application_Receipt_Date', 'Manager_DOJ'], axis=1)
test_X['Manager_Join_Year'] = test_X['Manager_Join_Year'].fillna( test_X.Manager_Join_Year.dropna().mean()).astype(np.int)
test_X['Manager_Join_Month'] = test_X['Manager_Join_Month'].fillna( test_X.Manager_Join_Month.dropna().mean()).astype(np.int)

test_X['Applicant_Marital_Status'] = test_X['Applicant_Marital_Status'].fillna( test_X.Applicant_Marital_Status.dropna().mode().values[0])


test_X['Office_PIN'] = le.fit_transform(test_X['Office_PIN'])

In [15]:
test_X['Applicant_Qualification'] = test_X['Applicant_Qualification'].fillna( test_X.Applicant_Qualification.dropna().mode().values[0])

In [16]:
test_X_std = sle.transform(test_X)
test_y= repredict(forest, .42, test_X_std)

In [17]:
df_sub = df_test

In [18]:
df_sub['Business_Sourced'] = test_y

In [19]:
df_sub = df_sub.drop(['Office_PIN', 'Application_Receipt_Date', 'Applicant_City_PIN', 'Applicant_Gender', 'Applicant_BirthDate', 
                      'Applicant_Marital_Status', 'Applicant_Occupation', 'Applicant_Qualification', 'Manager_DOJ',
                      'Manager_Joining_Designation', 'Manager_Current_Designation', 'Manager_Grade', 'Manager_Status',
                      'Manager_Gender', 'Manager_DoB', 'Manager_Num_Application', 'Manager_Num_Coded', 'Manager_Business',
                      'Manager_Num_Products', 'Manager_Business2', 'Manager_Num_Products2'], axis=1)

In [20]:
df_sub.to_csv('my_submission.csv', index=False)